In [9]:
from pm4py import view_ocdfg

from pybeamline.objects.ocdfg import OCDFG
from pybeamline.algorithms.oc.oc_dfg_merge_operator import oc_dfg_merge_operator
from pybeamline.algorithms.oc.oc_dfg_operator import oc_dfg_operator
from pybeamline.sources.ocel_log_source_from_file import ocel_log_source_from_file
from reactivex import operators as ops
from pm4py.ocel import OCEL
from pm4py.read import read_ocel2_json
from testfile import emitted_ocdfgs
from pm4py.algo.discovery.ocel.ocdfg import algorithm as ocdfg_discovery
## Test Script for Testing MVP3
# Comparing to process models by Jaccard similarity


# PM4Py
def read_ocel2_log(filename: str):
    return read_ocel2_json(filename)

oc_event_log = read_ocel2_log("../../tests/logistics.jsonocel")
assert oc_event_log is not None

# Do OCDFG discovery
ocdfg = ocdfg_discovery.apply(oc_event_log)
print(ocdfg.keys())
#view_ocdfg(ocdfg)
print(ocdfg["activities"])
print(ocdfg["edges"])



"""
# Log
log = ocel_log_source_from_file("../../tests/logistics.jsonocel")

# Set based
def jaccard_similarity(model: set, ref_model: set) -> float:

    Calculate the Jaccard similarity between two sets of model components.

    Parameters:
    model (set): Set of components in the first model.
    ref_model (set): Set of components in the reference model.

    Returns:
    float: Jaccard similarity coefficient.

    intersection = len(model.intersection(ref_model))
    union = len(model.union(ref_model))

    if union == 0:
        return 0.0  # Avoid division by zero

    return intersection / union

emitted_ocdfgs = []
def append_ocdfg(oc_dfg: OCDFG):
    global emitted_ocdfgs
    emitted_ocdfgs.append(oc_dfg)


log.pipe(
    oc_dfg_operator(),
    oc_dfg_merge_operator()
).subscribe(lambda oc_dfg: append_ocdfg(oc_dfg))

print(len(emitted_ocdfgs))
"""

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



'\n# Log\nlog = ocel_log_source_from_file("../../tests/logistics.jsonocel")\n\n# Set based\ndef jaccard_similarity(model: set, ref_model: set) -> float:\n\n    Calculate the Jaccard similarity between two sets of model components.\n\n    Parameters:\n    model (set): Set of components in the first model.\n    ref_model (set): Set of components in the reference model.\n\n    Returns:\n    float: Jaccard similarity coefficient.\n\n    intersection = len(model.intersection(ref_model))\n    union = len(model.union(ref_model))\n\n    if union == 0:\n        return 0.0  # Avoid division by zero\n\n    return intersection / union\n\nemitted_ocdfgs = []\ndef append_ocdfg(oc_dfg: OCDFG):\n    global emitted_ocdfgs\n    emitted_ocdfgs.append(oc_dfg)\n\n\nlog.pipe(\n    oc_dfg_operator(),\n    oc_dfg_merge_operator()\n).subscribe(lambda oc_dfg: append_ocdfg(oc_dfg))\n\nprint(len(emitted_ocdfgs))\n'